**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
# !pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout
import collections
import random
from collections import deque

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        '''
        self.epsilon = epsilon
        self.n_action = n_action
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        '''
        pass

    def act(self, act):
        '''
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a
        '''
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
        
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act() gives the action that the agent will do next. 

Without epsilon, the agent will always choose the best action leant from before, even there is a better option at the moment. With epsilon, the agent has a probability of epsilon that it acts randomly. This allows it to develop better actions.  Epsilon is used to balance exploration and exploitation.  




***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

```

In [51]:
'''
epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0

    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        try: 
          state, reward, game_over = env.act(action)
        except: 
          print('error')
        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
    '''

error


NameError: ignored

error


NameError: ignored

In [0]:
# print(env.act(action))

None


# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Array 'position' indicates the position of the mouse. All elements are 0 except the the element that indicates the position of the mouse is 1. 

Array 'board' indicates the board of the game. The values represents the reward of each position, either positive, zero or negative. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epochs):
        
        ##### FILL IN HERE        
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        
        print("Win/lose count {:.2f}/{:.2f}. Average score ({:.2f})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(round(score/epochs,2)))

In [76]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.00/7.00. Average score (-1.00)
Win/lose count 9.00/7.00. Average score (0.50)
Win/lose count 6.00/6.00. Average score (0.33)
Win/lose count 3.00/7.00. Average score (-0.75)
Win/lose count 5.50/8.00. Average score (-1.10)
Win/lose count 4.00/10.00. Average score (-1.92)
Win/lose count 5.00/4.00. Average score (-1.50)
Win/lose count 6.00/7.00. Average score (-1.44)
Win/lose count 4.00/10.00. Average score (-1.94)
Win/lose count 7.00/10.00. Average score (-2.05)
Win/lose count 3.00/9.00. Average score (-2.41)
Win/lose count 3.50/5.00. Average score (-2.33)
Win/lose count 2.00/13.00. Average score (-3.00)
Win/lose count 4.50/10.00. Average score (-3.18)
Win/lose count 3.50/12.00. Average score (-3.53)
Win/lose count 6.50/9.00. Average score (-3.47)
Win/lose count 4.00/7.00. Average score (-3.44)
Win/lose count 3.50/3.00. Average score (-3.22)
Win/lose count 5.00/5.00. Average score (-3.05)
Win/lose count 1.50/2.00. Average score (-2.92)
Final score: -2.92


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Please find the answer of this question in an uploaded file named 'DL3Q5.jpg'. 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        self.memory = collections.deque(self.memory, maxlen=self.max_memory)
        

    def random_access(self):
        return random.choice(self.memory)
        

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        act_pred = self.model.predict(np.reshape(s, (1, 5, 5, self.n_state)))
        
        return np.argmax(act_pred[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            batch=self.memory.random_access()
            input_states[i]=batch[0] # state observed in t; batch[1] is the state in t+1
            game_over_=batch[4]
            if game_over_:
                ######## FILL IN
                target_q[i, batch[2]] = batch[3]
            else:
                ######## FILL IN
                x = np.reshape(batch[1], (1, 5, 5, self.n_state)) # get observered state in t+1
                target_q[i, batch[2]] = batch[3] + self.discount * np.amax(self.model.predict(x))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(24, input_dim=50, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(4, activation='linear')) # keep four or set var = 4?
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [77]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/020 | Loss 0.0032 | Win/lose count 2.5/8.0 (-5.5)
Epoch 001/020 | Loss 0.0095 | Win/lose count 8.0/8.0 (0.0)
Epoch 002/020 | Loss 0.0102 | Win/lose count 6.5/10.0 (-3.5)
Epoch 003/020 | Loss 0.0093 | Win/lose count 1.5/2.0 (-0.5)
Epoch 004/020 | Loss 0.0042 | Win/lose count 3.0/6.0 (-3.0)
Epoch 005/020 | Loss 0.0009 | Win/lose count 2.0/3.0 (-1.0)
Epoch 006/020 | Loss 0.0222 | Win/lose count 4.5/3.0 (1.5)
Epoch 007/020 | Loss 0.0081 | Win/lose count 2.5/5.0 (-2.5)
Epoch 008/020 | Loss 0.0029 | Win/lose count 5.5/7.0 (-1.5)
Epoch 009/020 | Loss 0.0074 | Win/lose count 1.5/9.0 (-7.5)
Epoch 010/020 | Loss 0.0111 | Win/lose count 3.0/4.0 (-1.0)
Epoch 011/020 | Loss 0.0059 | Win/lose count 4.0/5.0 (-1.0)
Epoch 012/020 | Loss 0.0091 | Win/lose count 5.5/7.0 (-1.5)
Epoch 013/020 | Loss 0.0009 | Win/lose count 5.5/4.0 (1.5)
Epoch 014/020 | Loss 0.0045 | Win/lose count 1.5/5.0 (-3.5)
Epoch 015/020 | Loss 0.0124 | Win/lose count 5.5/2.0 (3.5)
Epoch 016/020 | Loss 0.0155 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        filters = 32
        kernel_size = (2,2)
        
        model = Sequential()
        model.add(Conv2D(filters, kernel_size, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        #print(model.summary())
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [78]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(grid_size = size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
# HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0101 | Win/lose count 5.0/5.0 (0.0)
Epoch 001/020 | Loss 0.0183 | Win/lose count 4.5/5.0 (-0.5)
Epoch 002/020 | Loss 0.0042 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/020 | Loss 0.0129 | Win/lose count 6.5/1.0 (5.5)
Epoch 004/020 | Loss 0.0053 | Win/lose count 7.0/6.0 (1.0)
Epoch 005/020 | Loss 0.0075 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/020 | Loss 0.0144 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/020 | Loss 0.0078 | Win/lose count 3.5/0 (3.5)
Epoch 008/020 | Loss 0.0037 | Win/lose count 5.0/3.0 (2.0)
Epoch 009/020 | Loss 0.0079 | Win/lose count 6.0/7.0 (-1.0)
Epoch 010/020 | Loss 0.0078 | Win/lose count 3.0/3.0 (0.0)
Epoch 011/020 | Loss 0.0054 | Win/lose count 2.0/0 (2.0)
Epoch 012/020 | Loss 0.0006 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/020 | Loss 0.0035 | Win/lose count 3.0/4.0 (-1.0)
Epoch 014/020 | Loss 0.0007 | Win/lose count 1.0/3.0 (-2.0)
Epoch 015/020 | Loss 0.0068 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/020 | Loss 0.0023 | Win/lose count 1.0/0 (1.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [79]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(grid_size = size, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN') 
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 7.50/3.00. Average score (4.50)
Win/lose count 2.50/3.00. Average score (2.00)
Win/lose count 7.50/4.00. Average score (2.50)
Win/lose count 5.50/5.00. Average score (2.00)
Win/lose count 1.00/1.00. Average score (1.60)
Win/lose count 7.00/4.00. Average score (1.83)
Win/lose count 2.50/4.00. Average score (1.36)
Win/lose count 2.00/5.00. Average score (0.81)
Win/lose count 2.50/2.00. Average score (0.78)
Win/lose count 4.50/1.00. Average score (1.05)
Win/lose count 5.00/6.00. Average score (0.86)
Win/lose count 3.00/2.00. Average score (0.88)
Win/lose count 8.50/3.00. Average score (1.23)
Win/lose count 7.50/4.00. Average score (1.39)
Win/lose count 11.50/6.00. Average score (1.67)
Win/lose count 5.50/4.00. Average score (1.66)
Win/lose count 6.50/3.00. Average score (1.76)
Win/lose count 4.00/1.00. Average score (1.83)
Win/lose count 5.00/4.00. Average score (1.79)
Win/lose count 5.00/3.00. Average score (1.80)
Final score: 1.8
Test of the FC
Win/lose co

In [0]:
# HTML(display_videos('cnn_test10.mp4'))

In [0]:
# HTML(display_videos('fc_test10.mp4'))

CNN agent performs better than FC agent. One issue is that there is volatility in the test results. Also, epsilon doesn't change which means as the agent learns he doesn't reduce the proportion of randomness in his game.

The temperature corresponds to the parmater of the binomial distribution which determines the density of poison and cheese on the board. The higher the temperature, the higher the proportion of bonus and so the easier the game. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Add an epsilon
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))
        
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))

        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        
        # Add a malus position
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
    ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [81]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(grid_size = size, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
# HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0080 | Win/lose count 2.0/6.819999999999964 (-4.819999999999964)
Epoch 001/020 | Loss 0.0027 | Win/lose count 3.98/4.89999999999996 (-0.9199999999999604)
Epoch 002/020 | Loss 0.0049 | Win/lose count 3.5/2.7899999999999987 (0.7100000000000013)
Epoch 003/020 | Loss 0.0026 | Win/lose count 1.49/1.9500000000000015 (-0.4600000000000015)
Epoch 004/020 | Loss 0.0130 | Win/lose count 1.97/5.909999999999979 (-3.939999999999979)
Epoch 005/020 | Loss 0.0082 | Win/lose count 1.49/2.939999999999981 (-1.4499999999999809)
Epoch 006/020 | Loss 0.0004 | Win/lose count 1.98/1.9400000000000015 (0.03999999999999848)
Epoch 007/020 | Loss 0.0022 | Win/lose count 0.98/1.9700000000000015 (-0.9900000000000015)
Epoch 008/020 | Loss 0.0026 | Win/lose count 0.5/1.9900000000000015 (-1.4900000000000015)
Epoch 009/020 | Loss 0.0022 | Win/lose count 1.48/3.9499999999999806 (-2.4699999999999807)
Epoch 010/020 | Loss 0.0002 | Win/lose count 0.5/2.97999999999998 (-2.47999999999998)
Epoch 011/020 |

In [82]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
# HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 2.96/1.93. Average score (1.03)
Win/lose count 0.00/2.01. Average score (-0.49)
Win/lose count 0.99/2.98. Average score (-0.99)
Win/lose count 0.49/2.00. Average score (-1.12)
Win/lose count 3.92/2.92. Average score (-0.70)
Win/lose count 1.47/1.98. Average score (-0.66)
Win/lose count 0.98/1.99. Average score (-0.71)
Win/lose count 3.48/2.93. Average score (-0.56)
Win/lose count 0.98/2.97. Average score (-0.72)
Win/lose count 0.49/1.99. Average score (-0.79)
Win/lose count 0.98/1.99. Average score (-0.81)
Win/lose count 4.93/8.91. Average score (-1.08)
Win/lose count 0.49/2.00. Average score (-1.11)
Win/lose count 0.00/2.99. Average score (-1.24)
Win/lose count 0.00/2.00. Average score (-1.30)
Win/lose count 2.94/1.95. Average score (-1.15)
Win/lose count 1.47/2.98. Average score (-1.17)
Win/lose count 0.00/2.00. Average score (-1.22)
Win/lose count 0.00/2.00. Average score (-1.26)
Win/lose count 2.96/3.95. Average score (-1.25)
Final score: -1.25


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***